# Lecture et visualisation rapide et validation des dataframe

In [ ]:
# cahrgeons les modules et packages python nécessaire
import os
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

exécutons votre script

In [ ]:
%run create_dataframe.py

Commençons par lire le fichier _data_nb_points.csv_

In [ ]:
# à vous

Nous allons observer la répartition des points avec une carte de chaleur du module Seaborn : https://seaborn.pydata.org/examples/spreadsheet_heatmap.html

In [ ]:
DATA_NB_PTS = pd.pivot_table(DATA_NB_POINTS,
                             values='nb_points',
                             index=['name'],
                             columns=['num_essai'])
DATA_NB_PTS

In [ ]:
f, ax = plt.subplots(figsize=(25, 15))
sns.heatmap(DATA_NB_PTS, annot=True, linewidths=.5, ax=ax);

Nous retrouvons clairement la distinction des essais lors de la phase d'entrainement et lors de la phase de test.
Nous nous attendons à 3000 points pour les essais en phase d'entrainement.

Affichez la carte de chaleur pour les 12 premiers essais des différences à 3000 points attendus.

In [ ]:
NB_POINTS_ENT = ... # regarder la méthode apply sur dataframe
NB_POINTS_ENT.head()

Appliquons de même la méthode pivot_table puis affichons la carte de chaleur

In [ ]:
NB_PTS_ENT = pd.pivot_table(NB_POINTS_ENT,
                             values='nb_points',
                             index=['name'],
                             columns=['num_essai'])
f, ax = plt.subplots(figsize=(25, 15))
sns.heatmap(NB_PTS_ENT, annot=True, linewidths=.5, ax=ax);

Faites de même pour le nombre de points lors de la phase de test pour lequel nous attendons 7500 points. 

In [ ]:
NB_POINTS_TEST = ...
NB_PTS_TEST = pd.pivot_table(...)
f, ax = plt.subplots(figsize=(25, 15))
sns.heatmap(NB_PTS_TEST, annot=True, linewidths=.5, ax=ax);

Que pouvons-nous en conclure ?

Regardons maintenant le contrebalancement des sujets et qui sont-ils ?

Pour ce faire :
- Chargez le fichier data_static.zip
- Affichez le nom des sujets pour chaque cas
- Affichez leur nombre

In [ ]:
DATA_STATIC = pd.read_csv("data_static.zip", index_col=0)
print('sujets commençant par "pe" en contrôle "pos"')
print(...)
print(...)
print('-------------')
print('sujets commençant par "pc" en contrôle "pos"')
print(...)
print(...)
print('-------------')
print('sujets commençant par "pe" en contrôle "vit"')
print(...)
print(...)
print('-------------')
print('sujets commençant par "pc" en contrôle "vit"')
print(...)
print(...)

Nous reviendrons plus tard sur ce dataframe. Observons plutôt les fichiers qui nous demande attention.

Pour ce faire, chargeons les données du dataframe _data_tempo.zip_

In [ ]:
DATA_TEMPO = pd.read_csv("data_tempo.zip", index_col=0)

Je vous propose d'observer de manière interactive l'ensemble des fichiers. Nous aurons besoin principalement du module ipywidgets : https://ipywidgets.readthedocs.io/en/stable/examples/Using%20Interact.html

In [ ]:
from ipywidgets import interact
from ipywidgets import interactive
from ipywidgets import fixed
from ipywidgets import interact_manual

import ipywidgets as widgets

from matplotlib import pyplot as plt
import matplotlib as mpl
mpl.rcParams['xtick.major.size'] = 6.0

In [ ]:
# création des sliders pour modifier
num_sujet = widgets.IntSlider(min=1, max=22, step=1,
                              value=1,
                              description='num_sujet')

# les bornes du graphiques
num_essai = widgets.IntSlider(min=0, max=19, step=1,
                              value=0,
                              description='num_essai')

# agencement des sliders
ui = widgets.HBox([num_sujet, num_essai])

def update_plot(pos_ct, time, num_essai, pos_ct_interp):
    '''
    maj plot
    '''
    nb_point = 3000 if num_essai < 12 else 7500
    duree = 90 if num_essai < 12 else 150
    ma_fig = plt.figure(figsize=(20, 16))
    plt.subplot(321)
    plt.plot(pos_ct[1:])
    plt.xlim([0, nb_point])
    plt.subplot(323)
    plt.plot(time[1:], pos_ct[1:])
    plt.xlim([time[1], time[-1]])
    plt.subplot(325)
    plt.plot(pos_ct_interp.values)
    plt.xlim([0, nb_point])
    plt.subplot(322)
    v_theo = np.diff(np.array(pos_ct[1:])) / 0.02
    v_pratique = np.diff(np.array(pos_ct[1:])) / np.diff(np.array(time[1:]))
    plt.xlim([0, nb_point])
    plt.plot(v_pratique - v_theo, 'r')
    plt.ylim([-500, 500])
    plt.subplot(324)
    plt.plot(np.diff(np.array(time)))
    plt.xlim([0, nb_point])
    plt.subplot(326)
    v_interp = np.diff(pos_ct_interp.dropna().values[1:]) / 0.02
    plt.plot(v_interp[:len(v_theo)] - v_theo, 'r')
    plt.xlim([0, nb_point])
    plt.ylim([-500, 500])
    return ma_fig

def visualisation_pos_ct(num_sujet=1, num_essai=0):
    """
    mu est la moyenne
    sigma est l'erreur standard
    tps_s est la durée du signal
    fe est la fréquence d'échantillonage
    """
    essai = !ls ../../data/ALL_DATA/sujet{num_sujet}/*_{num_essai}.json
    with open(essai[0], 'r') as fnm:
        contenu = json.load(fnm)
    update_plot(contenu['pos_ct'],
                contenu['routine_time'],
                num_essai,
                DATA_TEMPO['pos_ct_' + os.path.basename(essai[0])[:-5]])

out = widgets.interactive_output(visualisation_pos_ct, {'num_sujet': num_sujet,
                                                        'num_essai': num_essai})

display(ui, out)

Qu'en dites vous ? On garde, on ne garde pas ? On supprime l'essai qui dérange, on supprime le sujet ?

## Chargeons nos deux DataFrame et observons leurs contenus

Commençons par DATA_TEMPO qui est déjà chargé

In [ ]:
DATA_TEMPO

Nous remarquons que tous les essais ne sont pas de la même durée. Nous savons que les essais en phase d'entrainement ne durent qu'une minute et que les essais en phase de test durent 2min30.

Première étape, regardons les colonnes dédiées à l'entrainement et supprimons les lignes dans lesquelles nous y avons des np.nan.

In [ ]:
DATA_TEMPO_ENT = DATA_TEMPO.filter(regex='ent').dropna()
DATA_TEMPO_ENT

Commençant à 0ms, il est normal que nous obtenions une fin à 59,980s.
Nous pouvons toutefois s'assurer qu'aucune itération n'a été éliminée en validant que 3000 iterations de 20ms correspond bien à notre index.

In [ ]:
pd.date_range(start='7/1/2021', periods=3000, freq='20ms')

Nous pouvons réaliser la même chose pour la phase de test.

In [ ]:
DATA_TEMPO_TEST = ...
DATA_TEMPO_TEST

In [ ]:
pd.date_range(...)

Nous n'avons pas les 2min30 escomptées mais 2min 29,96 secondes.

Sur la base de ces deux clarifications, nous pouvons reconstruire un DATA_TEMPO plus homogène à travers tous les essais.

- Utiliser la méthode "join" et regarder l'option "how"

In [ ]:
DATA_TEMPO = ...
DATA_TEMPO

Il nous reste plus qu'à regarder si la colonne fb_err_* est bien la différence entre reco_data_* - pos_ct_* à chaque instant. C'est `fb_err` qui était présenté en poursuite d'erreur. `fb_err`était à annuler.

$$ fbErr_i = recoData_i - posCt_{i}$$

In [ ]:
# extraire les noms des colonnes au format list où le mot clé fb_err est présent
fb_err = ...
fb_err

In [ ]:
# tester l'agalité entre les colonnes dont le nom contient "fb_err" et la soustraction des colonnes associées à "reco_data" - "pos_ct"
...

La différence ne semble pas constament égale ! Comment est-ce possible ?

In [ ]:
# Au lieu de tester l'égalité regardons la différence !
...

C'est peut-être pas égale mais la différence est à 10e-14 ou c'est quand il y a des np.nan !
Calculons la différence en prenant en compte une tolérance et les Not A Number grâce à la méthode `allclose` de numpy.

In [ ]:
...

In [ ]:
for each_fb_err in fb_err:
    each_reco_data = each_fb_err.replace('fb_err', 'reco_data')
    each_pos_ct = each_fb_err.replace('fb_err', 'pos_ct')
    print(np.allclose(DATA_TEMPO[each_fb_err], DATA_TEMPO[each_reco_data] - DATA_TEMPO[each_pos_ct],
                      equal_nan=True))

In [ ]:
list(filter(lambda x: not np.allclose(DATA_TEMPO[x],
                                      (DATA_TEMPO[x.replace('fb_err', 'reco_data')] -
                                       DATA_TEMPO[x.replace('fb_err', 'pos_ct')]),
                                      equal_nan=True),
            fb_err))

C'est validé ! Notre Dataframe est prêt à nous aider pour la suite.

Regardons maintenant notre second dataframe.

## Data Static

In [ ]:
#Charger en mémoir en mémoire data_static
DATA_STATIC = ...
DATA_STATIC

Combien de sujets avons-nous ? Combien de sujets ont réalisés les modes de contrôle position et vitesse ?

- Utiliser la méthode unique

In [ ]:
NB_SUJET_CTRL_POS = ...
NB_SUJET_CTRL_VIT = ...
print(f'le nombre de sujet est de : {len(DATA_STATIC.name.unique())}')
print(f'le nombre de sujet pour le mode contrôle position est de : {NB_SUJET_CTRL_POS}')
print(f'le nombre de sujet pour le mode contrôle vitesse est de : {NB_SUJET_CTRL_VIT}')

Les sujets ont dus tous forcer et travailler à 22% de leur force maximale. La force des sujets est-elle bien répartie sur ces deux modes de contrôle ?

In [ ]:
# visualisation de la distribution du 22%MVF en fonction des sujets qui ont participé en contrôle position vs contrôle vitesse
REDUCE_DATA = DATA_STATIC[...]  # ne récupérons que les colonnes qui nous intéressent name, ctrl et force_level
UNIQUE_REDUCE_DATA = ...  # retire les duplication

plt.figure(figsize=(15, 7))
...  # trouver un plot dans la gallerie du module seaborn qui pourrait être une bonne représentation
plt.show()

Qu'en pensez-vous ?

Continuons d'explorer et appliquons la fonction de description de notre DataFrame et observons

In [ ]:
DATA_STATIC.describe()

Cette méthode `describe` ne s'applique que sur des colonnes à données numériques.  
La colonne `num_essai` ou `cutoff` n'a pas de sens ici. Nous aurions dû traiter ces deux colonnes comme des paramètres au format chaine de caractères.  
La colonne `force_level` et `percent_err` nous donne des informations à travers tous les sujets de toutes les conditions.

In [ ]:
DATA_STATIC['num_essai'] = DATA_STATIC['num_essai']  # appliquer sur cette colonne la méthode de conversion en chaine de caractère
DATA_STATIC['cutoff'] = DATA_STATIC['cutoff']  # faites de même

In [ ]:
# avons-nous bien travaillé ?
DATA_STATIC.describe()

# Remplacer num_essai et cutoff en chaine de caractère

Nous pouvons valider l'homogénéité des conditions à travers tous les sujets avant de commencer à les comparer.

Ont-ils tous le même nombre d'essai ?  
Ont-ils tous le même nombre d'essai `test` et `ent` ?  
Ont-ils tous le même nombre d'essai aux différentes valeurs de fréquence de coupure ?  
Ont-ils tous le même nombre d'essai en poursuite d'erreur `pe` et en poursuite de cible`pc` ?  

In [ ]:
# retrouvons nous les 22 sujets ?
sorted(DATA_STATIC.name.unique())

In [ ]:
# retrouvons nous les 20 essais de 0 à 19
...

In [ ]:
# regarder la méthode "value_counts"
...

In [ ]:
# Faites de même pour la phase de test
...

In [ ]:
# puis la phase d'entrainement
...

In [ ]:
# ne pouvons-nous pas aller plus loin avec la comptabilisation par groupe ?
# regarder la méthode count appliquée sur la méthode groupy
DATA_STATIC.groupby(['cutoff']).count()

In [ ]:
# retrouver toutes les valeurs de force_level
...

In [ ]:
# appliquer la méthode count sur le groupe cutoff et phase
...

In [ ]:
# appliquer la méthode count sur le groupe "ctrl", "tracking", "phase" et "cutoff"
...

Il ne nous reste plus qu'à enregistrer nos nouveaux DataFrame nettoyés !

In [ ]:
compression_opts = dict(method='zip',
                        archive_name='data_static.csv')
DATA_STATIC.to_csv(os.path.join('.', 'jour3', 'data_static.zip'),
                   compression=compression_opts)

compression_opts = dict(method='zip',
                        archive_name='data_tempo.csv')
DATA_TEMPO.to_csv(os.path.join('.', 'data_tempo.zip'),
                   compression=compression_opts)